In [53]:
import json
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser

In [54]:
# Lade Eingabedaten
with open("data/5.4.2_RE/sätze.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

In [55]:
# Neue Entity- und Relationstypen
entity_types = ['person', 'location', 'organisation', 'date']

relation_types = [
    # Biografische Basisdaten
    "geboren_in",      # Person – Ort
    "geboren_am",      # Person – Datum
    "gestorben_in",    # Person – Ort
    "gestorben_am",    # Person – Datum
    "begraben_in",     # Person – Ort

    # Wohn- und Aufenthaltsorte
    "wohnhaft_in",     # Person – Ort
    "aufenthalt_in",   # Person – Ort

    # Bildung und Beruf
    "ausgebildet_in",  # Person – Ort/Organisation
    "studierte_an",    # Person – Organisation
    "tätig_in",        # Person – Ort
    "tätig_bei",       # Person – Organisation
    "tätig_als",       # Person – Rolle/Funktion (Freitext)

    # Familie
    "verheiratet_mit", # Person – Person
    "kind_von",        # Person – Person
    "eltern_von",      # Person – Person

    # Mitgliedschaft
    "mitglied_von"     # Person – Organisation
    ]

In [56]:
class ExtractedInfo(BaseModel):
    subjekt: str = Field(description="extrahierte Subjekt-Entität, z.B. Daniel Wilhelm Suhl, ich, er, wir, meine l. Frau, seliger Bruder")
    subjekt_type: str = Field(description="Typ der Subjekt-Entität, z.B. person")
    prädikat: str = Field(description="Beziehung zwischen Subjekt und Objekt")
    objekt: str = Field(description="extrahierte Objekt-Entität, z.B. Gnadenthal, Aeltesten-Konferenz")
    objekt_type: str = Field(description="Typ der Objekt-Entität, z.B. location, organisation")
    zeit: str = Field(default=None, description="Datumsangabe zum Ereignis, z.B. 30 April 1858 oder leer, falls nicht genannt")

In [57]:
# Parser mit neuem Modell
parser = PydanticOutputParser(pydantic_object=ExtractedInfo)

In [58]:
# Systemprompt
system_prompt = PromptTemplate(
    template="""
Du arbeitest mit biographischen, historischen und religiösen Texten der Herrnhuter Brüdergemeine.  
Deine Aufgabe ist es, **Relationen** zwischen Personen, Orten, Organisationen und Daten zu extrahieren.  

Gib **ausschließlich** eine **JSON-Liste** mit Objekten zurück. Jedes Objekt enthält:  
- "subjekt"  
- "subjekt_type"  
- "prädikat"  
- "objekt"  
- "objekt_type"  
- "zeit"  

**Erlaubte Entitätstypen:** person, location, organisation, date  

**Erlaubte Prädikate (Kanon):**  
– „wohnte, lebte, ansässig in“ → wohnhaft_in
– „hielt sich auf, reiste nach, kam an, verließ“ → aufenthalt_in
– „arbeitete, wirkte, diente, beschäftigte sich, half, bekam den Antrag zu dienen“ → tätig_in (Ort) oder tätig_bei (Organisation)
– „studierte, lernte, hörte Vorlesungen“ → studierte_an
– „unterrichtet, ausgebildet, lernte Handwerk“ → ausgebildet_in
– „verheiratet mit, meine Frau, sein Mann“ → verheiratet_mit
– „Sohn von, Tochter von, Geschwister von“ → kind_von oder eltern_von
– „Mitglied, Bruder, Schwester, gehörte zur Bruderschaft“ → mitglied_von
– „berufen, gesandt, beauftragt, Ruf bekommen“ → tätig_in (Ort) / tätig_bei (Organisation)
- "entſchlafenen" → gestorben_in (Ort)/ gestorben_am (Datum)

**Regeln:**  
- Verwende **nur** die angegebenen Entitäten. Keine neuen Entitäten erfinden.  
- Synonyme intern auf die Kanon-Prädikate mappen.  
- Pronomen/Bezüge („ich“, „mir“, „mein(e) …“, „er“, „sie“, „wir“, „meine Frau“, „sein Vater“) dürfen im Output erscheinen.  
- Falls ein Datum genannt wird, setze es in `zeit` im ISO-Format (YYYY-MM-DD oder YYYY-MM oder YYYY).  
- Ein Objekt pro Relation. Keine zusammengesetzten Objekte.  
- Wenn keine Relation erkennbar ist, gib `[]` zurück.  
- Keine Erklärungen oder Fließtext.  

**Beispiele:**

Input-Satz: "Ich wurde am 29 September 1808 in Gnadenthal geboren."  
Entitäten:  
personen: Johann Adolf Bonatz  
orte: Gnadenthal  
date: 29 September 1808  
organisationen: –  

Output:
[
  {{
    "subjekt": "Johann Adolf Bonatz",
    "subjekt_type": "person",
    "prädikat": "geboren_in",
    "objekt": "Gnadenthal",
    "objekt_type": "location",
    "zeit": "1808-09-29"
  }},
  {{
    "subjekt": "Johann Adolf Bonatz",
    "subjekt_type": "person",
    "prädikat": "geboren_am",
    "objekt": "29 September 1808",
    "objekt_type": "date",
    "zeit": "1808-09-29"
  }}
]

---

Input-Satz: "…wo meine Eltern, Johann Gottlieb Bonaß und Johanna Dorothea geb. Koch, dienten."  
Entitäten:  
personen: Johann Adolf Bonatz, Johann Gottlieb Bonaß, Johanna Dorothea  
orte: Gnadenthal  
date: –  
organisationen: –  

Output:
[
  {{
    "subjekt": "Johann Adolf Bonatz",
    "subjekt_type": "person",
    "prädikat": "kind_von",
    "objekt": "Johann Gottlieb Bonaß",
    "objekt_type": "person",
    "zeit": null
  }},
  {{
    "subjekt": "Johann Adolf Bonatz",
    "subjekt_type": "person",
    "prädikat": "kind_von",
    "objekt": "Johanna Dorothea",
    "objekt_type": "person",
    "zeit": null
  }}
]
**Sonderregeln für Pronomen + Organisation + Ort:**

1. Pronomen („sie“, „er“, „ich“, „wir“) müssen immer auf eine Person in `personen` aufgelöst werden. 
   – Subjekt darf niemals leer sein.  
   – Wenn kein passender Name in `personen` steht → Relation verwerfen.

2. Sätze mit „die Gemeine in [Ort] bedienen / dienen / helfen“:
   – Subjekt = die Personen aus `personen` (jede einzeln).
   – Prädikat = „tätig_in“ (für den Ort) und zusätzlich „tätig_bei“ (für die Organisation).
   – Objekt = der Ort (`objekt_type = "location"`) und die Gemeine (`objekt_type = "organisation"`).

3. Wenn der Ort doppelt genannt wird („Mamre, damals Groenekloof“):
   – Nutze **Mamre** als Hauptort (`tätig_in`).
   – Optional kannst du zusätzlich ein zweites Triple mit `objekt = "Groenekloof"` erzeugen.

4. Zeitangaben wie „im Jahre 1810“ → `zeit = "1810"` (ISO-Format für Jahresangaben).

5. Beispiel:

Input-Satz:  
"Im Jahre 1810 bekamen sie den Antrag, die Gemeine in Mamre, damals Groenekloof, bedienen zu helfen."

Entitäten:  
personen: Johann Gottlieb Bonaß, Johanna Dorothea  
orte: Mamre, Groenekloof  
organisationen: Gemeine Mamre  
date: 1810  

Erwartete Ausgabe:
[
  {{
    "subjekt": "Johann Gottlieb Bonaß",
    "subjekt_type": "person",
    "prädikat": "tätig_in",
    "objekt": "Mamre",
    "objekt_type": "location",
    "zeit": "1810"
  }},
  {{
    "subjekt": "Johanna Dorothea",
    "subjekt_type": "person",
    "prädikat": "tätig_in",
    "objekt": "Mamre",
    "objekt_type": "location",
    "zeit": "1810"
  }},
  {{
    "subjekt": "Johann Gottlieb Bonaß",
    "subjekt_type": "person",
    "prädikat": "tätig_bei",
    "objekt": "Gemeine Mamre",
    "objekt_type": "organisation",
    "zeit": "1810"
  }},
  {{
    "subjekt": "Johanna Dorothea",
    "subjekt_type": "person",
    "subjekt_type": "person",
    "prädikat": "tätig_bei",
    "objekt": "Gemeine Mamre",
    "objekt_type": "organisation",
    "zeit": "1810"
  }}
]


""",
    input_variables=["entity_types", "relation_types"]
)
system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt)

In [59]:
# Humanprompt
human_prompt = PromptTemplate(
    template="""
Satz: "{text}"  

Entitäten im Satz:  
- personen: {personen}  
- orte: {orte}  
- date: {date}  
- organisationen: {organisationen}  

**Aufgabe:**  
Finde sinnvolle Relationen zwischen den angegebenen Entitäten.  
Nutze dabei nur die erlaubten Prädikate des Kanons.  
Wenn ein Datum vorhanden ist, übertrage es in `zeit`.  
Gib **nur** die JSON-Liste zurück, ohne Erklärungen.  

{format_instructions}
""",
    input_variables=["text", "personen", "orte", "date", "organisationen"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

In [60]:
# Chatmodell + Prompt-Kette
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatOllama(model="llama3", temperature=0.7)
chain = LLMChain(llm=model, prompt=chat_prompt)

In [61]:
results = []

# Extraktion starten
for entry in input_data:
    result = chain.run(
        entity_types=entity_types,
        relation_types=relation_types,
        text=entry["text"],
        personen=", ".join(entry.get("personen", [])),
        orte=", ".join(entry.get("orte", [])),
        date=", ".join(entry.get("date", [])),
        organisationen=", ".join(entry.get("organisationen", []))
    )
    # hier gleich transformieren, kein triple.json
    try:
        relations = json.loads(result)
    except json.JSONDecodeError:
        relations = []

    transformed = {
        "text": entry.get("text", ""),
        "ref": entry.get("ref", ""),
        "datei": entry.get("datei", ""),
        "autor": entry.get("autor", ""),
        "graph": relations
    }
    results.append(transformed)
    
    
# Speichere Ergebnisse
with open("data/5.4.2_RE/triple.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)    
